<a href="https://colab.research.google.com/github/lucid0465/-/blob/master/%5B%EC%8B%A4%EC%8A%B5_1%5D_Fraud_Detection_MLP_Modeling_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle 신용카드 사기 검출 
https://www.kaggle.com/mlg-ulb/creditcardfraud
## Credit Card Fraud Detection
* creditcard.csv (284,807 * 31)
* Class : <font color = 'blue'>'0' (정상결제)</font>, <font color = 'red'>'1' (부정결제)</font>
* 사기 검출(Fraud Detection), 이상 탐지(Anomaly Detection)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# I. wget From Github
* 'creditCardFraud.zip' 파일 다운로드

In [ ]:
!wget https://raw.githubusercontent.com/rusita-ai/pyData/master/creditCardFraud.zip

--2023-03-26 23:54:30--  https://raw.githubusercontent.com/rusita-ai/pyData/master/creditCardFraud.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155672 (66M) [application/zip]
Saving to: ‘creditCardFraud.zip’

creditCardFraud.zip 100%[===================>]  65.95M   279MB/s    in 0.2s    

2023-03-26 23:54:34 (279 MB/s) - ‘creditCardFraud.zip’ saved [69155672/69155672]



* 다운로드 결과 확인

In [ ]:
!ls -l 

total 67540
-rw-r--r-- 1 root root 69155672 Mar 26 23:54 creditCardFraud.zip
drwxr-xr-x 1 root root     4096 Mar 23 18:57 sample_data


# II. Data Preprocessing

> ## 1) Unzip 'creditCardFraud.zip'

* Colab 파일시스템에 'creditcard.csv' 파일 생성

In [ ]:
!unzip /content/creditCardFraud.zip

Archive:  /content/creditCardFraud.zip
  inflating: creditcard.csv          


* creditcard.csv 파일 확인

In [ ]:
!ls -l

total 214836
-rw-r--r-- 1 root root 150828752 Sep 20  2019 creditcard.csv
-rw-r--r-- 1 root root  69155672 Mar 26 23:54 creditCardFraud.zip
drwxr-xr-x 1 root root      4096 Mar 23 18:57 sample_data


> ## 2) 데이터 읽어오기

* pandas DataFrame

In [ ]:
import pandas as pd

DF = pd.read_csv('creditcard.csv')

DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
DF.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


* 0 (정상) Class와 1 (사기) Class 개수

In [ ]:
DF.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

* 0 (정상) Class와 1 (사기) Class 비율

In [ ]:
DF.Class.value_counts(normalize = True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

> ## 3) 'Time' -> 'hours'

* 'Time': 각 거래와 첫 번째 거래 사이에 경과된 초('Seconds') 

> ### (1) 시간('hours') 정보 생성

In [ ]:
timedelta = pd.to_timedelta(DF['Time'], unit = 's')

DF['Time'] = (timedelta.dt.components.hours).astype(int)

In [ ]:
DF.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


> ## 4) train_test_split( )

* X (Input), y (Output) 지정

In [ ]:
X = DF.iloc[:,:-1]
y = DF.iloc[:, -1]

X.shape, y.shape

((284807, 30), (284807,))

* With 'Stratify'

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 2045,
                                                    stratify = y)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((199364, 30), (199364,), (85443, 30), (85443,))

* Train_Data와 Test_Data의 1 (부정) 비율이 균형

In [ ]:
print('Train_Data :','\n', y_train.value_counts(normalize = True))
print()
print('Test_Data :','\n', y_test.value_counts(normalize = True))

Train_Data : 
 0    0.998275
1    0.001725
Name: Class, dtype: float64

Test_Data : 
 0    0.998268
1    0.001732
Name: Class, dtype: float64


# III. MLP Modeling

> ## 1) MLPClassifier Modeling

* hidden_layer_sizes : 은닉층 노드의 개수
* activation : 활성화 함수
* solver : 최적화 기법
* max_iter : 학습 반복 횟수

In [ ]:
from sklearn.neural_network import MLPClassifier

Model_NN = MLPClassifier(hidden_layer_sizes = (12),
                         activation = 'logistic',
                         solver ='adam', 
                         max_iter = 5000,
                         random_state = 2045)

Model_NN.fit(X_train, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=12, max_iter=5000,
              random_state=2045)

> ## 2) Test_Data에 Model 적용

In [ ]:
y_hat = Model_NN.predict(X_test)

> ## 3) Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_hat, labels = [1, 0])

array([[  123,    25],
       [   24, 85271]])

> ## 4) Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_hat, 
                            target_names = ['정상', '부정'],
                            digits = 5))

              precision    recall  f1-score   support

          정상    0.99971   0.99972   0.99971     85295
          부정    0.83673   0.83108   0.83390       148

    accuracy                        0.99943     85443
   macro avg    0.91822   0.91540   0.91681     85443
weighted avg    0.99942   0.99943   0.99943     85443



# IV. MLP Modeling with SMORT


> ## 1) SMOTE

* Before SMOTE

In [ ]:
X_train.shape, y_train.shape

((199364, 30), (199364,))

In [ ]:
pd.Series(y_train).value_counts()

0    199020
1       344
Name: Class, dtype: int64

* imbalanced-learn Package

In [ ]:
from imblearn.over_sampling import SMOTE 

* After SMOTE

In [ ]:
OS = SMOTE(random_state = 2045)

X_train_OS, y_train_OS = OS.fit_resample(X_train, y_train)



In [ ]:
X_train_OS.shape, y_train_OS.shape

((398040, 30), (398040,))

* 0 (정상) Class와 1 (사기) Class 개수

In [ ]:
pd.Series(y_train_OS).value_counts()

0    199020
1    199020
Name: Class, dtype: int64

> ## 2) MLPClassifier Modeling with SMOTE

* 약 2분

In [ ]:
%%time 

from sklearn.neural_network import MLPClassifier

Model_OS = MLPClassifier(hidden_layer_sizes = (16),
                         activation = 'logistic',
                         solver ='adam', 
                         max_iter = 5000,
                         random_state = 2045)

Model_OS.fit(X_train_OS, y_train_OS)

CPU times: user 1min 40s, sys: 316 ms, total: 1min 40s
Wall time: 1min 41s


MLPClassifier(activation='logistic', hidden_layer_sizes=16, max_iter=5000,
              random_state=2045)

> ## 3) Test_Data에 Model 적용

In [ ]:
y_hat = Model_OS.predict(X_test)

> ## 4) Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_hat, labels = [1, 0])

array([[  132,    16],
       [  212, 85083]])

> ## 5) Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_hat, 
                            target_names = ['정상', '부정'],
                            digits = 5))

              precision    recall  f1-score   support

          정상    0.99981   0.99751   0.99866     85295
          부정    0.38372   0.89189   0.53659       148

    accuracy                        0.99733     85443
   macro avg    0.69177   0.94470   0.76762     85443
weighted avg    0.99874   0.99733   0.99786     85443



# 
# 
# 
# The End
# 
# 
# 